In [11]:
from elasticsearch import Elasticsearch

client = Elasticsearch("https://localhost:9200/", basic_auth=("elastic", "_YnyYSU1F5mv9esOU70a"), ca_certs= "./http_ca.crt")
client.info()

ObjectApiResponse({'name': 'ba8d4d04b891', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'VfPIV9u9R2ekihOu9mX1SQ', 'version': {'number': '8.15.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '98adf7bf6bb69b66ab95b761c9e5aadb0bb059a3', 'build_date': '2024-09-19T10:06:03.564235954Z', 'build_snapshot': False, 'lucene_version': '9.11.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [20]:
import json 
resp = client.search(
    index="my-index",
    query={
        "multi_match": {
            "query": "Located on the coast of Maine, Acadia features rugged shorelines, granite peaks, and diverse habitats, making it a popular destination for outdoor enthusiasts.",
            "fields": ["title","description"]
        }
  },
)
resp["hits"]["hits"]

[{'_index': 'my-index',
  '_id': 'tBtidZIBFtksUa6aCRS_',
  '_score': 28.41713,
  '_source': {'id': 'park_acadia',
   'title': 'Acadia National Park',
   'description': 'Located on the coast of Maine, Acadia features rugged shorelines, granite peaks, and diverse habitats, making it a popular destination for outdoor enthusiasts.'}},
 {'_index': 'my-index',
  '_id': 'sBtidZIBFtksUa6aCRS_',
  '_score': 3.1548824,
  '_source': {'id': 'park_yosemite',
   'title': 'Yosemite National Park',
   'description': 'Famous for its breathtaking granite cliffs, waterfalls, and giant sequoias, Yosemite offers a diverse range of ecosystems and stunning vistas.'}},
 {'_index': 'my-index',
  '_id': 'shtidZIBFtksUa6aCRS_',
  '_score': 1.7026049,
  '_source': {'id': 'park_yellowstone',
   'title': 'Yellowstone National Park',
   'description': "Home to the world's first national park, Yellowstone is known for its geysers, hot springs, and diverse wildlife, including bison and grizzly bears."}},
 {'_index': '